In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.metrics import categorical_crossentropy
from keras.applications import Xception
from keras.applications.xception import preprocess_input

In [7]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1/255)       # initialise train generator
validate_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1/255)    # initialise validate generator
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1/255)        # initialise test generator

train_generator=train_datagen.flow_from_directory('images/train',         # get train images from directory
                                                 target_size=(114,114),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)
validation_generator = validate_datagen.flow_from_directory(             # get validation images from directory
        'images/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical',shuffle=True)

test_generator = test_datagen.flow_from_directory('images/test',         # get test images from directory
                                                 target_size=(114,114),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 1400 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [15]:
pretrained_model = Xception(weights='imagenet',input_shape = (114,114,3),include_top=False)   # load MobileNet with weights from ImageNet without the last layer
pretrained_model.trainable = False                                   # make the base model untrainable
x = pretrained_model.output                                          # take the output layer
x = GlobalAveragePooling2D()(x)                                      # add a pooling layer
x = Dense(1024,activation='relu')(x)                                 # add a dense layer
x = Dense(1024,activation='relu')(x)                                 # add a dense layer
#x = Dense(512,activation='relu')(x)                                  # add a dense layer
#x = Dropout(rate=0.1)(x)
pred_layer = Dense(2,activation='softmax')(x)                        # add a softmax layer
classification_model = Model(inputs=pretrained_model.input,outputs=pred_layer)       #initialise the model
classification_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])                                    # compile the model
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy
for layer in classification_model.layers[:133]:                       # set some layers as not trainable
    layer.trainable=False
for layer in classification_model.layers[133:]:                        # set the rest of the layers as trainable           
    layer.trainable=True
#print(len(classification_model.layers))
step_size_train=train_generator.n//train_generator.batch_size        # calculate the step size
history = classification_model.fit(                                  # train the model
        train_generator,
        steps_per_epoch=step_size_train,
        epochs=50,
        validation_data=validation_generator)
#classification_model.save('DL_model.h5')


Epoch 1/50
43/43 [==============================] - 110s 2s/step - loss: 0.6951 - accuracy: 0.4903 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/50
43/43 [==============================] - 119s 3s/step - loss: 0.6933 - accuracy: 0.4945 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/50
43/43 [==============================] - 136s 3s/step - loss: 0.6930 - accuracy: 0.5169 - val_loss: 0.6928 - val_accuracy: 0.5167
Epoch 4/50
43/43 [==============================] - 138s 3s/step - loss: 0.6928 - accuracy: 0.5158 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 5/50
43/43 [==============================] - 139s 3s/step - loss: 0.6933 - accuracy: 0.4968 - val_loss: 0.6915 - val_accuracy: 0.5500
Epoch 6/50
43/43 [==============================] - 138s 3s/step - loss: 0.6911 - accuracy: 0.5319 - val_loss: 0.6898 - val_accuracy: 0.5100
Epoch 7/50
43/43 [==============================] - 133s 3s/step - loss: 0.6882 - accuracy: 0.5384 - val_loss: 0.6885 - val_accuracy: 0.5033
Epoch 8/50
43

KeyboardInterrupt: 

In [11]:
classification_model.save('DL_model3.h5')
classification_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 114, 114, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 56, 56, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 56, 56, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 56, 56, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [14]:
classification_model = load_model('DL_model3.h5')  # load the classification model
classification_model.evaluate(train_generator)      # evaluate the model   

44/44 [==============================] - 61s 1s/step - loss: 0.6299 - accuracy: 0.6664


[0.6298882365226746, 0.6664285659790039]

In [12]:
train_generator.class_indices

{'Cat': 0, 'Dog': 1}

In [21]:
classification_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 114, 114, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 56, 56, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 56, 56, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 56, 56, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________